In [ ]:
%env OPENAI_API_KEY=<PUT_YOUR_API_KEY_HERE>


In [3]:
import os
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

audio_file= open("./data/WSJ1202210302.mp3", "rb")
transcript = client.audio.transcriptions.create(model="whisper-1", file=audio_file)
print(transcript.text)


Welcome to TechNews Briefing. It's Wednesday, November 29th. I'm Julie Chang for The Wall Street Journal. With holiday shopping in full force, one place you could turn to for gift ideas is generative AI chatbots like Chachipiti or Google's BARD. Cordelia James from our personal tech team will join us later in the show to discuss how to use AI to make your shopping list. Just be sure to check it twice. But first, starting with some headlines. Adobe's $20 billion acquisition of collaboration software Figma is in trouble. The UK Competition and Markets Authority said Tuesday it has provisionally found the deal would likely harm innovation by eliminating competition between two main companies in product design software. An Adobe spokesperson said the company disagrees with the CMA's perspective and that the company will re-engage with the regulator on the case. Figma also said it disagrees with the CMA's findings and that it remains committed to the deal. The regulator said it would listen

In [8]:
transcript = client.audio.transcriptions.create(model="whisper-1", file=audio_file, response_format="srt")
print(transcript)


1
00:00:00,000 --> 00:00:05,760
Welcome to TechNews Briefing.

2
00:00:05,760 --> 00:00:07,480
It's Wednesday, November 29th.

3
00:00:07,480 --> 00:00:10,040
I'm Julie Chang for The Wall Street Journal.

4
00:00:10,040 --> 00:00:16,440
With holiday shopping in full force, one place you could turn to for gift ideas is generative

5
00:00:16,440 --> 00:00:20,800
AI chatbots like Chachipiti or Google's BARD.

6
00:00:20,800 --> 00:00:25,240
Cordelia James from our personal tech team will join us later in the show to discuss

7
00:00:25,240 --> 00:00:28,600
how to use AI to make your shopping list.

8
00:00:28,600 --> 00:00:33,400
Just be sure to check it twice.

9
00:00:33,400 --> 00:00:36,160
But first, starting with some headlines.

10
00:00:36,160 --> 00:00:41,680
Adobe's $20 billion acquisition of collaboration software Figma is in trouble.

11
00:00:41,680 --> 00:00:46,600
The UK Competition and Markets Authority said Tuesday it has provisionally found the deal

12
00:00:46,600 --> 

In [10]:
audio_file = open("./data/nsf665-itunes.mp3", "rb")
transcript = client.audio.translations.create(
  model="whisper-1",
  file=audio_file)
print(transcript.text)


We are on Thursday, November 23, 2023. Welcome to our weekly news show, The News in Slow French. Hello everyone. Hello Arnaud. Hello Catherine. Hello to all our listeners. We will start our show with a discussion about the application of sanctions on Russian oil workers sailing in the Baltic Sea. Then we will talk about events organized in France on the occasion of the release of the Hollywood epic Napoleon. Our subject of scientific discussion of the week will then focus on a study that reveals that magicians are less subject to personality disorders than other artists. And to conclude this first part of the show, we will talk about the new Beaujolais 2023, which is available worldwide. The film Napoleon is a hit. I hope its release will be an opportunity to address Napoleon's controversial history and to discuss his legacy. Have you already seen it? Are you kidding Catherine? You and I, we are not ready to get tickets right away. It's true, but let's continue our announcements. What 

In [11]:
from pydub import AudioSegment

podcast = AudioSegment.from_mp3("./data/Download_on_Tech_with_Tris_Warkentin_.mp3")

# PyDub handles time in milliseconds
ten_minutes = 15 * 60 * 1000

total_length = len(podcast)

start = 0
index = 0
while start < total_length:
    end = start + ten_minutes
    if end < total_length:
        chunk = podcast[start:end]
    else:
        chunk = podcast[start:]
    with open(f"./data/clips/podcast_clip_{index}.mp3", "wb") as f:
        chunk.export(f, format="mp3")
    start = end
    index += 1


In [ ]:
for i in range(index):
    clip = f"./data/clips/podcast_clip_{i}.mp3"
    audio_file= open(clip, "rb")
    transcript = client.audio.transcriptions.create(model="whisper-1", file=audio_file)
    # mkdir ./data/transcripts if not exists
    if not os.path.exists("./data/transcripts"):
        os.makedirs("./data/transcripts")
    # write to file
    with open(f"./data/transcripts/podcast_clip_{i}.txt", "w") as f:
        f.write(transcript.text)
    # get last sentence of the transcript
    sentences = transcript.text.split(".")


In [1]:
import whisper

model = whisper.load_model("medium")
index = 2 # number of files

def transcript(clip, output):
    result = model.transcribe(clip)
    with open(output, "w") as f:
        f.write(result['text'])
    print("Transcripted: ", clip)

for i in range(index):
    clip = f"./data/clips/podcast_clip_{i}.mp3"
    output = f"./data/whisper_model_transcripts/podcast_clip_{i}.txt"
    transcript(clip, output)
    # get last sentence of the transcript
    with open(output, "r") as f:
        transcript_text = f.read()
    sentences = transcript_text.split(".")


Transcripted:  ./data/clips/podcast_clip_0.mp3
Transcripted:  ./data/clips/podcast_clip_1.mp3


In [18]:
from langchain.chat_models import ChatOpenAI
from llama_index import GPTListIndex, LLMPredictor, ServiceContext, SimpleDirectoryReader
from llama_index.node_parser import SentenceSplitter

# define LLM
llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-4", max_tokens=1024))

parser = SentenceSplitter(chunk_size=2048)
documents = SimpleDirectoryReader('./data/transcripts').load_data()
nodes = parser.get_nodes_from_documents(documents)

service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)

list_index = GPTListIndex(nodes=nodes, service_context=service_context)
index = list_index.as_query_engine(response_mode="tree_summarize")
response = index.query("Could you please provide me with the details or key points of your podcast? This will help me create an accurate and concise summary in English.")
print(response)


The podcast features Tris Workington, the Director of Product Management at Google DeepMind. The discussion revolves around the advancements in AI technology, its potential benefits, and the importance of responsible regulation. Tris highlights the role of AI in various sectors, including science, medicine, and climate change. He also discusses Google's AI products, such as Google Bard and Duet AI, and their potential to improve daily tasks like email and document creation. Tris emphasizes the importance of collaboration between AI developers and policymakers to ensure the safe and beneficial deployment of AI. He also mentions Google's commitment to the White House's voluntary commitments for responsible AI deployment. The podcast concludes with Tris expressing his hope for a future where AI can create significant societal impact and change.
